In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch import nn, Tensor

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

import random
import math
import time
import sys
sys.path.insert(1, '../code/')

import ssm
import vae
from vae import TransformerAutoencoder
# insert at 1, 0 is the script path (or '' in REPL)
import util

import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from torch.utils.data import dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


### Set the path and specification that you want to predict

In [5]:
data_path = '../data_full/transformer_data'
spec = 'airworthy' #'hover'#'interference'#'dist' #'mass' #'airworthy'
max_mass = 35. #(kg)

### Download the dataset and build the data loaders

In [7]:
torch.manual_seed(0)
np.random.seed(0)
batch_size = 16
batch_size_val = 16
frac_train = 0.4
frac_val = 0.4

dataloader_tr, dataloader_val, dataloader_test, scale_1, scale_2 = vae.prepare_sequence_data(data_path, spec, batch_size = batch_size ,batch_size_val = batch_size_val, frac_train = frac_train, frac_val = frac_val)

dataloader_tr.dataset.x_train = torch.clip(dataloader_tr.dataset.x_train, min=0, max = 1)
dataloader_val.dataset.x_train = torch.clip(dataloader_val.dataset.x_train, min=0, max = 1)
dataloader_test.dataset.x_train = torch.clip(dataloader_test.dataset.x_train, min=0, max = 1)
print(f'Training Data:   {dataloader_tr.dataset.y_train.shape[0]}')
print(f'Validation Data: {dataloader_val.dataset.y_train.shape[0]}')
print(f'Test Data:       {dataloader_test.dataset.y_train.shape[0]}')
print(f'Mean of Training Data: {torch.mean(dataloader_tr.dataset.x_train)}')
print(f'Var of training Data: {torch.var(dataloader_tr.dataset.x_train)}')
print(f'Min of training Data: {torch.min(dataloader_tr.dataset.x_train)}')
print(f'Max of training Data: {torch.max(dataloader_tr.dataset.x_train)}')

Training Data:   11085
Validation Data: 11085
Test Data:       5544
Mean of Training Data: 0.0010118670761585236
Var of training Data: 0.0009689386934041977
Min of training Data: 0.0
Max of training Data: 1.0


In [8]:
def decode_embedding(x):
    ''' 
    x is one embedding sample [69(timestep), 653(embedding size)]
    '''
    def decode(dic, tokens, values):
        '''
        Take an encoding dictionary {'token0':0, 'token1':1, ... } and list of tokens [0, 1, 4, ... ]
        and return the corresponding strings of tokens ['token0', 'token1', 'token4', ...]
        '''
    
        seq = []
        dic_reverse = {value:key for key, value in dic.items()}
        for n, tok in enumerate(tokens):
            if dic_reverse[int(tok)] == 'Value':
                seq.append(float(values[n]))
            else:
                seq.append(dic_reverse[int(tok)])
        return seq
    data_path = '../data/transformer_data'
    dic = torch.load(data_path)
    K = len(dic['encoding_dict_keys'])
    V = len(dic['encoding_dict_values'])
    k = decode(dic['encoding_dict_keys'], x[:,:K].argmax(1), None)
    v = decode(dic['encoding_dict_values'], x[:,K:K+V].argmax(1), dic['X_norm'][-6][:,-1])
    if 20 in list(x[:,:K].argmax(1)):
        end_idx = list(x[:,:K].argmax(1)).index(20) #20 # why it is sometimes 29 and sometimes 20? k.index('varioY') #end token
    else:
        end_idx = len(list(x[:,:K].argmax(1)))
    k = k[:end_idx+1]
    v = v[:end_idx+1]
    print(x[:,:K].argmax(1), x[:,K:K+V].argmax(1))
    output = [{k[i]:v[i]} for i in range(len(k))]
    return output
    

In [9]:
### Show input of autoencoder

In [10]:
input = dataloader_val.dataset.x_train[0]
print(input.size())
input_seq = decode_embedding(input)
print(input_seq, len(input_seq))


torch.Size([140, 749])
tensor([32, 32, 19, 42, 33, 32,  9, 40, 41,  7, 21,  3, 18, 26, 39,  2, 24, 22,
        12, 35, 37, 36,  0, 38, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]) tensor([489, 526,   0,   0, 415, 266, 477,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   

### Load pre-trained trasformer autoencoder model

In [11]:
torch.manual_seed(0)
np.random.seed(0)

emsize = 20  # embedding dimension
d_hid = 512  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 8  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 20  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
D_out = 20
D = dataloader_tr.dataset.x_train.shape[-1]
device = "cpu"

model = TransformerAutoencoder( emsize, nhead, d_hid, nlayers, dropout, D, D_out).to(device)
model=torch.load('../code/code/autoencoder_model_state.pth')
print(model.state_dict)


<bound method Module.state_dict of TransformerAutoencoder(
  (transformer_encoder): TransformerModel(
    (pos_encoder): PositionalEncoding(
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-7): 8 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=20, out_features=20, bias=True)
          )
          (linear1): Linear(in_features=20, out_features=512, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=512, out_features=20, bias=True)
          (norm1): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.2, inplace=False)
          (dropout2): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (encoder): Linear(in_features=749, out_features=20, bias=Tr

### Make prediction of autoencoder (validation data)

In [12]:
batch, test_data= next(enumerate(dataloader_test))
input, targets, mask = test_data
input, targets, mask =input[2:3], targets[2:3], mask[2:3]  
print(input.size())

torch.Size([1, 140, 749])


In [13]:
embedding, mem = model.transformer_encoder(input, mask)
print(embedding.size(), mem.size())
out  =model.transformer_decoder(embedding,mem,mask)
out = out.permute(1, 0, 2)
print(input.size(), out.size())

torch.Size([140, 1, 20]) torch.Size([140, 1, 20])
torch.Size([1, 140, 749]) torch.Size([1, 140, 749])


### Input sequence

In [14]:
input_seq = decode_embedding(input[0])
print(input_seq)
print(len(input_seq))

tensor([32, 32, 19, 42, 33, 17,  6, 32, 19, 42, 33, 17,  6, 32, 19, 42, 33, 32,
         9, 40, 41,  7, 21, 25, 23, 15,  1, 26, 39,  2, 24, 22, 12, 35, 37, 36,
         0, 38, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]) tensor([125, 239,   0, 159, 246,   0,   0, 239,   0,   0, 535,   0,   0, 526,
          0, 524, 415,  85, 371,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 

### Output Sequence

In [15]:
out_seq = decode_embedding(out[0])
print(out_seq)
print(len(out_seq))

tensor([32, 32, 19, 42, 42, 17, 19, 32, 19, 42, 42, 19,  6, 32, 19, 32, 42, 32,
        42,  6,  6,  6,  6, 19,  6,  6,  6, 35,  6, 35,  6, 35,  6, 35,  6, 35,
         6, 35,  6, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]) tensor([239, 239, 220, 193, 193, 239, 526, 239, 220, 193, 193, 526, 239, 239,
        220, 239, 193, 239, 239, 239, 239, 526, 239, 526, 239, 239, 526, 559,
        559, 526, 526, 526, 559, 526, 526, 559, 239, 526, 526, 239, 239, 239,
        239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239,
        239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 

### Make Prediction of training data

In [16]:
batch, test_data= next(enumerate(dataloader_tr))
input, targets, mask = test_data
input, targets, mask =input[10:11], targets[10:11], mask[10:11]  
print(input.size())

torch.Size([1, 140, 749])


In [17]:
embedding, mem = model.transformer_encoder(input, mask)
print(embedding.size(), mem.size())
out  =model.transformer_decoder(embedding,mem,mask)
out = out.permute(1, 0, 2)
print(input.size(), out.size())

torch.Size([140, 1, 20]) torch.Size([140, 1, 20])
torch.Size([1, 140, 749]) torch.Size([1, 140, 749])


### Input sequence

In [18]:
input_seq = decode_embedding(input[0])
print(input_seq)

tensor([32, 32,  6, 19, 32, 19, 42, 33, 17,  6, 32,  9, 40, 41,  7, 21, 25, 23,
        15,  1, 26, 39,  2, 24, 22, 12, 35, 37, 36,  0, 38, 20,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]) tensor([ 27, 482,   0,   0, 239,   0,  25, 559,   0,   0,  85,  63,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 

### Output Sequence

In [19]:
out_seq = decode_embedding(out[0])
print(out_seq)

tensor([32, 32,  6, 19, 32, 19, 42, 42, 17,  6, 32, 42,  6,  6,  6,  6,  6,  6,
         6,  6, 35,  6,  2,  6,  2,  6,  2,  6, 35,  6, 35,  6, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]) tensor([239, 239, 239, 220, 239, 220, 193, 193, 239, 559, 239, 239, 239, 239,
        239, 239, 559, 526, 239, 239, 559, 526, 559, 239, 559, 526, 559, 239,
        559, 526, 559, 559, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239,
        239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239,
        239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 